In [1399]:
import pandas as pd

In [1400]:
data_scrap = pd.read_csv('../../data/scrapped/avito_cars_scrapping.csv')

In [1401]:
#shape of our scrapped data
data_scrap.shape

(1302, 14)

In [1402]:
data_scrap.head()

,Date Pub,Ville,Année-Modèle,Boite de vitesses,Type de carburant,Kilométrage,Marque,Modèle,Nombre de portes,Origine,Première main,Puissance fiscale,État,Prix
0,il y a 2 heures,Casablanca,2018.0,Automatique,Essence,100 000 - 109 999,Fiat,500,5.0,WW au Maroc,Oui,7 CV,Excellent,107 000 DH
1,il y a 6 heures,Rabat,2022.0,Automatique,Diesel,120 000 - 129 999,BMW,X5,5.0,WW au Maroc,Oui,12 CV,Excellent,590 000 DH
2,il y a 6 heures,Marrakech,2014.0,Automatique,Diesel,120 000 - 129 999,Jeep,Grand Cherokee,5.0,Dédouanée,Non,12 CV,Excellent,165 000 DH
3,il y a 6 heures,Fès,2021.0,Manuelle,Diesel,90 000 - 94 999,Renault,Clio,5.0,WW au Maroc,Oui,6 CV,Excellent,128 000 DH
4,il y a 6 heures,Agadir,2020.0,Automatique,Diesel,140 000 - 149 999,Skoda,Karoq,5.0,Dédouanée,Oui,6 CV,Excellent,278 000 DH


In [1403]:
data_scrap.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1302 entries, 0 to 1301
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Date Pub           1302 non-null   object 
 1   Ville              1300 non-null   object 
 2   Année-Modèle       1298 non-null   float64
 3   Boite de vitesses  1298 non-null   object 
 4   Type de carburant  1298 non-null   object 
 5   Kilométrage        1298 non-null   object 
 6   Marque             1298 non-null   object 
 7   Modèle             1298 non-null   object 
 8   Nombre de portes   1164 non-null   float64
 9   Origine            1170 non-null   object 
 10  Première main      1158 non-null   object 
 11  Puissance fiscale  1263 non-null   object 
 12  État               1218 non-null   object 
 13  Prix               1302 non-null   object 
dtypes: float64(2), object(12)
memory usage: 142.5+ KB


In [1404]:
#number of duplicates
data_scrap.duplicated().sum()

239

In [1405]:
#drop duplicates
data_scrap.drop_duplicates(inplace=True)

In [1406]:
#convert the column Année-Modèle to a num
data_scrap['Année-Modèle'] = pd.to_numeric(data_scrap['Année-Modèle'], errors='coerce').fillna(0).astype(int)

In [1407]:
#remove the column Date_pub
data_scrap = data_scrap.drop(columns=['Date Pub','État'])

In [1408]:
#remove the CV in the column Puissange fiscale  
data_scrap['Puissance fiscale']=data_scrap['Puissance fiscale'].astype(str).str.split().apply(lambda x : x[0] if len(x[0])>0 else None)
#convert to the column to num
data_scrap['Puissance fiscale'] = pd.to_numeric(data_scrap['Puissance fiscale'], errors='coerce')

In [1409]:
#remove DH from price
data_scrap['Prix']=data_scrap['Prix'].astype(str).str.replace('DH','').str.replace(r'\s+','', regex=True)
#convert to the column to num
data_scrap['Prix'] = pd.to_numeric(data_scrap['Prix'], errors='coerce')

In [1410]:
#convert the column Kilometrage to a single value
data_scrap['Kilométrage'] = data_scrap['Kilométrage'].astype(str).str.replace(' ','').str.split('-').apply(lambda x :(int(x[0])+int(x[1]))/2 if len(x)==2 else None)

In [1411]:
#number of nan values in each column
data_scrap.isna().sum()

Ville                  2
Année-Modèle           0
Boite de vitesses      4
Type de carburant      4
Kilométrage            5
Marque                 4
Modèle                 4
Nombre de portes     118
Origine              115
Première main        119
Puissance fiscale     40
Prix                   0
dtype: int64

In [1412]:
#name of columns
data_scrap.columns.to_list()

['Ville',
 'Année-Modèle',
 'Boite de vitesses',
 'Type de carburant',
 'Kilométrage',
 'Marque',
 'Modèle',
 'Nombre de portes',
 'Origine',
 'Première main',
 'Puissance fiscale',
 'Prix']

In [1413]:
#remove the none values 
data_scrap = data_scrap.dropna(subset=['Année-Modèle','Boite de vitesses','Type de carburant','Kilométrage',
'Marque','Modèle','Origine','Première main','Puissance fiscale'])
data_scrapped.shape

(915, 12)

In [1414]:
#unique values of 
data_scrap['Type de carburant'].unique()

array(['Essence', 'Diesel', 'Hybride', 'Electrique'], dtype=object)

In [1415]:
# Liste des modèles ayant 3 portes
models_with_3_doors = data_scrap[data_scrap['Nombre de portes']==3]['Modèle'].unique()

# Remplir les valeurs manquantes pour ces modèles
data_scrap.loc[
    (data_scrap['Modèle'].isin(models_with_3_doors)) & 
    (data_scrap['Nombre de portes'].isna()),
    'Nombre de portes'
] = 3


In [1416]:
# Récupérer la liste des modèles avec 5 portes
models_with_5_doors = data_scrapped[data_scrapped['Nombre de portes']==5]['Modèle'].unique()

# Remplir les valeurs manquantes
data_scrap.loc[
    (data_scrap['Modèle'].isin(models_with_5_doors)) & 
    (data_scrap['Nombre de portes'].isna()),
    'Nombre de portes'
] = 5

In [1417]:
#remove the rows where theres left none values in nombre de portes
data_scrap=data_scrap.dropna(subset=['Nombre de portes'])

In [1418]:
data_scrap.shape

(916, 12)

In [1419]:
#see number of the rows where price <20000
data_scrap[data_scrap['Prix']<20000]

,Ville,Année-Modèle,Boite de vitesses,Type de carburant,Kilométrage,Marque,Modèle,Nombre de portes,Origine,Première main,Puissance fiscale,Prix
507,Casablanca,2006,Manuelle,Diesel,67499.5,Toyota,Hilux,3.0,Dédouanée,Non,12.0,11


In [1420]:
#remove all rows that has a weird price
data_scrapped=data_scrap.drop(data_scrap[data_scrap['Prix'] < 20000].index)

In [1421]:
data_scrap.shape

(916, 12)

In [1422]:
data_scrap=data_scrapped.drop(data_scrap[data_scrap['Origine']=='Pas encore dédouanée'].index)

In [1423]:
#convert the column Puissance 
data_scrap['Puissance fiscale']= data_scrap['Puissance fiscale'].astype(int)

In [1424]:
data_scrap['Origine'].unique()

array(['WW au Maroc', 'Dédouanée', 'Importée neuve'], dtype=object)

In [1425]:
data_scrap['Ville'].unique()

array(['Casablanca', 'Rabat', 'Marrakech', 'Fès', 'Agadir', 'Tétouan',
       'Agadir Melloul', 'Salé', 'Mohammedia', 'Meknès', 'Nador',
       'Tanger', 'Ifrane', 'Kénitra', 'Laâyoune', 'Ben Ahmed', 'Temara',
       'Dar Bouazza', 'Oujda', 'الدار البيضاء', 'Ouazzane', 'El Jadida',
       'Béni Mellal', 'Larache', 'Ben Guerir', 'Essaouira', 'Safi',
       'Ouarzazate', 'Khouribga', 'Sidi Rahal', 'القنيطرة',
       'Fquih Ben Saleh', 'الرباط', 'سلا'], dtype=object)

In [1426]:
data_scrap['Ville'] = data_scrap['Ville'].str.replace('الدار البيضاء', 'Casablanca')

In [1427]:
data_scrap['Ville'] = data_scrap['Ville'].str.replace('الرباط', 'Rabat')

In [1428]:
data_scrap['Ville']=data_scrap['Ville'].str.replace('سلا','Salé')

In [1429]:
data_scrap['Ville']=data_scrap['Ville'].str.replace('القنيطرة','Kénitra')

In [1430]:
#number of nan values in each column
data_scrap.isna().sum()

Ville                0
Année-Modèle         0
Boite de vitesses    0
Type de carburant    0
Kilométrage          0
Marque               0
Modèle               0
Nombre de portes     0
Origine              0
Première main        0
Puissance fiscale    0
Prix                 0
dtype: int64

In [1431]:
data_scrap.shape

(914, 12)

In [1432]:
data_scrap.to_csv('../../data/scrapped/voitures_clean2.csv',index=False)